# Start Up

Run the cells below. They will open a browser window through selenium. (check your open windows if you don't notice it. It will be a firefox page with an orange url bar.)

Log into your ESPN account and navigate to the draft.

Make sure to keep everything in a single tab. Do not open links as new windows, do not open links as new tabs. This should be the default behavior if you just click on things normally (but no promises - this is hacked together at best...)

### Updating the probabilities

The default score/pick probability info is for 10 team, standard (non-ppr) scoring.

You can update the probabilities with different parameters ```yahoowizard.create_prob_pickle(teams, outputname, mapping_path, scoring, outputpath)``` where teams is the number of teams in your league, and scoring is one of these four entries:
 - 'standard'
 - 'ppr'
 - 'half-ppr'
 - '2qb

The other parameters deal with file mappings.
 
Follow this link to see the source data [Fantasy Football Calculator](https://fantasyfootballcalculator.com/scenario-calculator)

### Updating/Changing the Projections

You can call ```yahoowizard.process_projections(projection_path, idmap_path, output_path)```
to update or change the projections. ```projection_path``` is the direction to the projection files. I use [fantasyfootballanalytics.net](https://www.fantasyfootballanalytics.net)

You can use any source of projections as long as you can map them to the ESPN IDs. 

Required columns:
 - 'position'
 - 'player'
 - 'points'
 - 'adp'
 
This needs more development, specifically:
 - mapping tables to cross-reference all the different sources to some sort of master ID
 - It would also be useful to map the player IDs to the different providers.
 - making the column naming more flexible 
 
 
 # Current Use.
 tells you how to use it... tbd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd 
import numpy as np
import yahoowizard as yw

# projections

In [ ]:
proj_name = 'yahoo_players.pkl'
prob_name = 'pick_prob_NEW_10_standard.pkl'

In [ ]:
yw.process_projections('yahoo_projections.csv',
                       'espn_ffa_map.csv',
                       proj_name)

In [ ]:
yw.create_prob_pickle(10,
                       'half-ppr',
                       prob_name,
                       'probmap.csv')

In [ ]:
projections = pd.read_pickle(proj_name)

probabilities = pd.read_pickle(prob_name)

In [ ]:
league_id = 123456789
your_first_pick = 4

In [ ]:
draft = yw.YahooDraft(league_id, your_first_pick)

In [ ]:
draft.league_id

In [ ]:
draft.first_pick

In [ ]:
draft.current_pick

In [ ]:
draft.num_teams

In [ ]:
oc = yw.tell_me_what_to_do(projections,probabilities,draft)
cols = ['player','team','position','points','oc_raw_0','oc_adj_0','pb_0','oc_raw_1','oc_adj_1','pb_1']
print('\nRoster Needs')
print(draft.pos_needs.loc[['QB','RB','WR','TE','DST','K']])
print('\n')
oc[cols].sort_values('oc_adj_0',ascending=False).head(10)

In [ ]:
projections = yw.blacklist_player('Leonard Fournette','JAC',projections)
# projections = yw.unblacklist_player('Leonard Fournette','JAC',projections)

In [ ]:
def next_picks(pick_order, mypick, thispick):
    next_pick = pick_order.index(mypick,thispick) + 1
    next_pick2 = pick_order.index(mypick,next_pick) + 1
    return next_pick, next_pick2

def adj_probs(players,probs, next_pick, next_pick2):
    players['oc_adj'] = players['need_adj']*1 + ~players['need_adj']*players['oc_mult']
    players['espn_id'] = players['espn_id'].astype(str)
    probs['espnid'] = probs['espnid'].astype(str)
    for i, pick in enumerate([next_pick,next_pick2]):
        pmask = probs['pick'] == pick
        df = players.merge(probs[pmask], how='left', left_on='espn_id', right_on='espnid')
        df['%'] = df['%'].fillna(1)
        df['%'] = np.where(df['picked'] | df['blacklist'], 0, df['%'])
        df['probpicked'] = 1 - df['%']
        df = df.groupby('position').apply(expected_max)
        palt = df.groupby('position')['emax'].sum()
        df['opp'] = df['position'].map(palt)
        players[f'oc_raw_{i}'] = (df['points'] - df['opp']) * df['probpicked'] * ~df['picked'] * ~df['blacklist']
        players[f'oc_adj_{i}'] = players[f'oc_raw_{i}'] * players['oc_adj']
        players[f'pb_{i}'] = df['probpicked']
    return players.copy()

In [ ]:
np1, np2 = next_picks(draft.pick_order, draft.my_team, draft.current_pick)

In [ ]:
np1, np2

In [ ]:
projections[projections['position']=='DST'][['player','team','espn_id']].sort_values('team')

In [ ]:
nfl = pd.read_json('nfl.json')

In [ ]:
nfl[nfl['position'] == 'DEF'].sort_index()

In [ ]:
def_map = pd.read_clipboard()

In [ ]:
def_map

In [ ]:
def_map = def_map.astype(str).set_index('index')

In [ ]:
nfl['mapping_val'] = nfl.index

In [ ]:
nfl['mapping_val']

In [ ]:
nfl['yahoo_id_def'] = nfl['mapping_val'].map(def_map['yahoo_id'])

In [ ]:
nfl['yahoo_id'] = nfl['yahoo_id'].fillna(nfl['yahoo_id_def'])

In [ ]:
nfl = nfl.drop(columns=['yahoo_id_def'])

In [ ]:
nfl.to

In [ ]:
draft.update(projections)

In [ ]:
draft.update(projections)

In [ ]:
draft.picked_players


In [ ]:
draft.picked_players

In [ ]:
from yahoo_oauth import OAuth2
oauth = OAuth2(None, None, from_file='oauth.json')
if not oauth.token_is_valid():
    oauth.refresh_access_token()

In [ ]:
r = oauth.session.get('https://fantasysports.yahooapis.com/fantasy/v2/league/nfl.l.1022820/players;player_keys=nfl.p.100032')

In [ ]:
print(r.text)

In [ ]:
draft.scrape_draft()

In [ ]:
draft.picked_players

In [ ]:
import xml.etree.ElementTree as ET 
import re

In [ ]:
players = {}
teams = draft.num_teams
for i in range(1,teams+1):
    _url = f"{draft.league_url}/teams;team_keys=nfl.l.{draft.league_id}.t.{i}/players"
    _r = oauth.session.get(_url)
    xmlstring = _r.text
    xmlstring = re.sub(' xmlns="[^"]+"', '', xmlstring, count=1)
    root = ET.fromstring(xmlstring)
    for player in root.iter('player_id'):
        players[player.text] = i

In [ ]:
players

In [ ]:
nfl = nfl.T

In [ ]:
nfl[nfl['position'] == 'DEF'][['player_id','espn_id','yahoo_id']]

In [ ]:
players['need_adj'].unique()

In [ ]:
players['need_adj']*1 + (~players['need_adj'])